<a href="https://colab.research.google.com/github/drupal786/Algos/blob/main/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, struct, pickle, json
from sys import getsizeof
from dataclasses import dataclass, field
from typing import Tuple, Dict

In [ ]:
instrument_tokens = {
    f"BANKNIFTY18SEP24C{i}": i*13 for i in range(42000, 52000, 100)
} | {f"BANKNIFTY18SEP24P{i}": i*19 for i in range(42000, 52000, 100)}

token_instruments = {
    v: k
    for k, v in instrument_tokens.items()
}

In [ ]:
raw_instrument_data = {
    'ft': 1726115101,
    'tk': '881600',
    'lp': 2.85,
    'ohlc': {
        'o': 2.85,
        'h': 2.9,
        'l': 2.85,
        'c': 2.85,
        'v': 2844810,
        'oi': 119460
    }
}
raw_200_instruments_data = {
    k: {} | raw_instrument_data
    for cp in (
        (f"BANKNIFTY18SEP24C{i}", f"BANKNIFTY18SEP24P{i}")
        for i in range(42000, 52000, 100)
    )
    for k in cp
}
[
    raw_200_instruments_data[k].__setitem__("tk", str(instrument_tokens[k]))
    for k in raw_200_instruments_data
]
# raw_200_instruments_data

In [ ]:
print(f"{getsizeof(raw_instrument_data) = } bytes") # 232 bytes as python dict object
print(f"{getsizeof(json.dumps(raw_instrument_data)) = } bytes") # 176 bytes as a string obtaine dby json.dumps method (For transmission over network as serilalised object)
print(f"{getsizeof(raw_200_instruments_data) = } bytes") # 9,312 bytes / 9.09375 KiB or python dict object
print(f"{getsizeof(json.dumps(raw_200_instruments_data)) = } bytes") # 31,049 bytes / 30.3213 KiB as a string obtaine dby json.dumps method (For transmission over network as serilalised object)

getsizeof(raw_instrument_data) = 232 bytes
getsizeof(json.dumps(raw_instrument_data)) = 176 bytes
getsizeof(raw_200_instruments_data) = 9312 bytes
getsizeof(json.dumps(raw_200_instruments_data)) = 31049 bytes


In [ ]:
@dataclass
class Ohlcvoi:
    open: float
    high: float
    low: float
    close: float
    volume: int
    oi: int

    @classmethod
    def from_dict(cls, raw: Dict[str, float|int]) -> "Ohlcvoi":
        return cls(*(
            raw[k]
            for k in (*"ohlcv", "oi")
        ))

    def as_dict(self) -> Dict[str, float|int]:
        return {
            "o": self.open,
            "h": self.high,
            "l": self.low,
            "c": self.close,
            "v": self.volume,
            "oi": self.oi
        }

    def as_bin(self) -> bytes:
        return struct.pack(">4f2I", *(self.open, self.high, self.low, self.close, self.volume, self.oi))

    @classmethod
    def from_bin(cls, data: bytes) -> "Ohlcvoi":
        return cls(*struct.unpack(">4f2I", data))

@dataclass
class InstrumentData:
    forward_time: int
    token: int
    last_price: float
    ohlc: Ohlcvoi

    @classmethod
    def from_dict(cls, raw: Dict[str, float|int]) -> "InstrumentData":
        return cls(
            raw["ft"],
            int(raw["tk"]),
            raw["lp"],
            Ohlcvoi.from_dict(raw["ohlc"])
        )

    def as_dict(self) -> Dict[str, float|int]:
        return {
            "ft": self.forward_time,
            "tk": str(self.token),
            "lp": self.last_price,
            "ohlc": self.ohlc.as_dict()
        }

    def as_bin(self) -> bytes:
        return (
            struct.pack(">2If", *(self.forward_time, self.token, self.last_price))
            + self.ohlc.as_bin()
        )

    @classmethod
    def from_bin(cls, data: bytes) -> "Ohlcvoi":
        return cls(
            *struct.unpack(">2If", data[:12]),
            Ohlcvoi.from_bin(data[12:])
        )

@dataclass
class InstrumentsData:
    data: Dict[str, InstrumentData] = field(default_factory=dict)

    @classmethod
    def from_dict(cls, raw: Dict[str, Dict[str, float|int]]) -> "InstrumentsData":
        return cls(
            {
                k: InstrumentData.from_dict(v)
                for k, v in raw.items()
            }
        )

    def as_dict(self) -> Dict[str, Dict[str, float|int]]:
        return {
            k: v.as_dict()
            for k, v in self.data.items()
        }

    def as_bin(self) -> bytes:
        return struct.pack(">I", len(self)) + b"".join(
            v.as_bin()
            for v in self.data.values()
        )

    @classmethod
    def from_bin(cls, data: bytes) -> "InstrumentsData":
        packet_len = struct.calcsize(">2I5f2I")
        no_of_packets = struct.unpack(">I", data[:4])[0]
        print(packet_len, no_of_packets)
        data = data[4:]
        dict_data = [
            InstrumentData.from_bin(data[i*packet_len:(i+1)*packet_len])
            for i in range(no_of_packets)
        ]
        return cls(
            {
                token_instruments[v.token]: v
                for v in dict_data

            }
        )

    def __contains__(self, key: str) -> bool:
        return key in self.data

    def __getitem__(self, key: str) -> InstrumentData:
        return self.data[key]

    def __setitem__(self, key: str, value: InstrumentData):
        self.data[key] = value

    def __len__(self) -> int:
        return len(self.data)

In [ ]:
Ohlcvoi.from_dict(raw_instrument_data["ohlc"])

Ohlcvoi(open=2.85, high=2.9, low=2.85, close=2.85, volume=2844810, oi=119460)

In [ ]:
InstrumentData.from_dict(raw_instrument_data)

InstrumentData(forward_time=1726115101, token=881600, last_price=2.85, ohlc=Ohlcvoi(open=2.85, high=2.9, low=2.85, close=2.85, volume=2844810, oi=119460))

In [ ]:
print(f"{getsizeof(InstrumentData.from_dict(raw_instrument_data)) = } bytes") # 48 bytes, while same data as python dict object takes 232 bytes

getsizeof(InstrumentData.from_dict(raw_instrument_data)) = 48 bytes


In [ ]:
print(InstrumentData.from_dict(raw_instrument_data).as_bin(), end="\n"*2)

b'f\xe2m\x1d\x00\rs\xc0@6ff@6ff@9\x99\x9a@6ff@6ff\x00+h\x8a\x00\x01\xd2\xa4'



In [ ]:
InstrumentsData.from_dict(raw_200_instruments_data)

In [ ]:
InstrumentsData.from_dict(raw_200_instruments_data).as_bin()

In [ ]:
InstrumentsData.from_bin(InstrumentsData.from_dict(raw_200_instruments_data).as_bin())

In [ ]:
print(f"{len(InstrumentsData.from_dict(raw_200_instruments_data).as_bin()) = } bytes")
# 36 * 200 = 7200 bytes / 7.03125 KiB Plus No Packets U32(4 bytes) Prefixed, Hence 7,204 bytes / 7.03515 KiB for 200 Instruments
# while same data serialized as string and ready for transmission over network takes 31,049 bytes / 30.3213 KiB as a string obtained by json.dumps method

print(f"{len(InstrumentData.from_dict(raw_instrument_data).as_bin()) = } bytes")
# 36 bytes (serialised as binary data, ready for transmission over network), while same data serialized as string and ready for transmission over network takes 176 bytes

len(InstrumentsData.from_dict(raw_200_instruments_data).as_bin()) = 7204 bytes
len(InstrumentData.from_dict(raw_instrument_data).as_bin()) = 36 bytes
